
# AB Testi ile Bidding Yöntemlerinin Dönüşümünün Karşılaştırılması

**İş Problemi**

Facebook kısa süre önce mevcut "maximumbidding" adı verilen teklif verme türüne alternatif
olarak yeni bir teklif türü olan "average bidding"’i tanıttı. Müşterilerimizden biri olan bombabomba.com,
bu yeni özelliği test etmeye karar verdi veaveragebidding'in maximumbidding'den daha fazla dönüşüm
getirip getirmediğini anlamak için bir A/B testi yapmak istiyor.A/B testi 1 aydır devam ediyor ve
bombabomba.com şimdi sizden bu A/B testinin sonuçlarını analiz etmenizi bekliyor. Bombabomba.com için
nihai başarı ölçütü Purchase'dır. Bu nedenle, istatistiksel testler için Purchase metriğine odaklanılmalıdır.

*Veri Seti Hikayesi*

Bir firmanın web site bilgilerini içeren bu veri setinde kullanıcıların gördükleri ve tıkladıkları
reklam sayıları gibi bilgilerin yanı sıra buradan gelen kazanç bilgileri yer almaktadır.Kontrol ve Test
grubu olmak üzere iki ayrı veri seti vardır. Bu veri setleriab_testing.xlsxexcel’ininayrı sayfalarında yer
almaktadır. Kontrol grubuna Maximum Bidding, test grubuna AverageBiddinguygulanmıştır.

- impression: Reklam görüntüleme sayısı
- Click: Görüntülenen reklama tıklama sayısı
- Purchase: Tıklanan reklamlar sonrası satın alınan ürün sayısı
- Earning: Satın alınan ürünler sonrası elde edilen kazanç


**Proje Görevleri**

> *AB Testing (Bağımsız İki Örneklem T Testi)*

1.   Hipotezleri Kur
2.   Varsayım Kontrolü
  - Normallik Varsayımı (shapiro)
  - Varyans Homojenliği (levene)
3. Hipotezin Uygulanması
  - Varsayımlar sağlanıyorsa bağımsız iki örneklem t testi
  - Varsayımlar sağlanmıyorsa mannwhitneyu testi
4. p-value değerine göre sonuçları yorumla

Not:
- Normallik sağlanmıyorsa direkt 2 numara. Varyans homojenliği sağlanmıyorsa 1 numaraya arguman girilir.
- Normallik incelemesi öncesi aykırı değer incelemesi ve düzeltmesi yapmak faydalı olabilir.

**Görev 1:  Veriyi Hazırlama ve Analiz Etme**

In [ ]:
# Adım 1:  ab_testing_data.xlsx adlı kontrol ve test grubu verilerinden oluşan veri setini okutunuz. Kontrol ve test grubu verilerini ayrı değişkenlere atayınız.
import pandas as pd
import numpy as np
from scipy.stats import shapiro, levene, ttest_ind

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

control_df = pd.read_excel("src/measurement_problems/ABTesti/ab_testing.xlsx", sheet_name="Control Group")
test_df = pd.read_excel("src/measurement_problems/ABTesti/ab_testing.xlsx", sheet_name="Test Group")


In [ ]:
# Adım 2: Kontrol ve test grubu verilerini analiz ediniz.
def missing_values_analysis(data):
    na_columns = [col for col in data.columns if data[col].isnull().sum() > 0]
    n_miss = data[na_columns].isnull().sum().sort_values(ascending=True)
    ratio = (data[na_columns].isnull().sum() / data.shape[0] * 100).sort_values(ascending=True)
    missing_df = pd.concat([n_miss, np.round(ratio, 2)], axis=1, keys=['Total Missing Values', 'Ratio'])
    missing_df = pd.DataFrame(missing_df)
    return missing_df

def check_dataframe(data, row_num=5):
    print("*************** Dataset Shape ***************")
    print("No. of Rows:", data.shape[0], "\nNo. of Columns:", data.shape[1])
    print("*************** Dataset Information ***************")
    print(data.info())
    print("*************** Types of Columns ***************")
    print(data.dtypes)
    print(f"*************** First {row_num} Rows ***************")
    print(data.head(row_num))
    print(f"*************** Last {row_num} Rows ***************")
    print(data.tail(row_num))
    print("*************** Summary Statistics of The Dataset ***************")
    print(data.describe().T)
    print("*************** Dataset Missing Values Analysis ***************")
    print(missing_values_analysis(data))



check_dataframe(control_df)
check_dataframe(test_df)


In [ ]:
# Adım 3: Analiz işleminden sonra concat metodunu kullanarak kontrol ve test grubu verilerini birleştiriniz.
test_df['group'] = 'test'
control_df['group'] = 'control'
df = pd.concat([control_df, test_df], axis=0, ignore_index=True)

df.head(5)
df.tail(5)


**Görev 2:  A/B Testinin Hipotezinin Tanımlanması**


In [ ]:
# Adım 1: Hipotezi tanımlayınız.
############################
# 1. Hipotezi Kur
############################

# H0: M1 = M2 satın alınan ürün sayısı arasında istatistiksel olarak anlamlı bir fark yoktur
# H1: M1 != M2 fark vardır

# Adım 2: Kontrol ve test grubu için purchase(kazanç) ortalamalarını analiz ediniz
df.groupby("group").agg({'Purchase':'mean'})


**GÖREV 3: Hipotez Testinin Gerçekleştirilmesi**


In [ ]:
######################################################
# AB Testing (Bağımsız İki Örneklem T Testi)
######################################################


# Adım 1: Hipotez testi yapılmadan önce varsayım kontrollerini yapınız.Bunlar Normallik Varsayımı ve Varyans Homojenliğidir.

# Kontrol ve test grubunun normallik varsayımına uyup uymadığını Purchase değişkeni üzerinden ayrı ayrı test ediniz

############################
# Normallik Varsayımı
############################

# H0: Normal dağılım varsayımı sağlanmaktadır.
# H1:..sağlanmamaktadır.
# p-value < 0.05'ten HO RED.
# p-value > 0.05 H0 REDDEDILEMEZ.

test_stat, pvalue = shapiro(df.loc[df["group"] == "control", "Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))
#Test Stat = 0.9773, p-value = 0.5891

test_stat, pvalue = shapiro(df.loc[df["group"] == "test", "Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))
#Test Stat = 0.9589, p-value = 0.1541

#her iki grubun p-value 0.05'ten büyük olduğu içi H0 reddedilemez

############################
# Varyans Homojenligi Varsayımı
############################

# H0: Varyanslar Homojendir
# H1: Varyanslar Homojen Değildir

test_stat, pvalue = levene(df.loc[df["group"] == "control", "Purchase"],
                           df.loc[df["group"] == "test", "Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))
#Test Stat = 2.6393, p-value = 0.1083
# p-value < 0.05 'ten HO RED.
# p-value > 0.05 H0 REDDEDILEMEZ.

#p-value değeri 0.05'ten büyük olduğu için H0 reddedilemez


In [ ]:
# Adım 2: Normallik Varsayımı ve Varyans Homojenliği sonuçlarına göre uygun testi seçiniz
# Varsayımlar sağlandığı için bağımsız iki örneklem t testi (parametrik test)
test_stat, pvalue = ttest_ind(df.loc[df["group"] == "control", "Purchase"],
                              df.loc[df["group"] == "test", "Purchase"],
                              equal_var=True)

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))
# Test Stat = -0.9416, p-value = 0.3493
# p-value < ise 0.05 'ten HO RED.
# p-value < değilse 0.05 H0 REDDEDILEMEZ.


In [ ]:
# Adım 3: Test sonucunda elde edilen p_value değerini göz önünde bulundurarak kontrol ve test grubu satın alma
# ortalamaları arasında istatistiki olarak anlamlı bir fark olup olmadığını yorumlayınız.

#p-value değeri 0.05'ten büyük olduğu için H0 reddedilemez
# satın alınan ürün sayısında istatistiksel olarak anlamlı bir farkın olmadığı gözlemlenebilmektedir


**GÖREV 4 : Sonuçların Analizi**

In [ ]:
# Adım 1: Hangi testi kullandınız, sebeplerini belirtiniz.
# Kontrol ve test grubunun normallik varsayımına uyup uymadığını test etmek için Shapiro testi uyguladık
# normallik varsayımı sağlandığı için varyans homojenliğini kontrol ettik
# İki grubun varyans homojenliği gözlemleyebilmek için Levene testi kullandık.
# test sonucu varyans homojenlikleri arasında benzerlik olduğu gördük
# iki grup arasındaki istatistiksel anlamları ortaya çıkarabilmek için Bağımsız İki Örneklem T Testi uyguladık


# Adım 2: Elde ettiğiniz test sonuçlarına göre müşteriye tavsiyede bulununuz.
# Test ve kontrol grupları arasında ortalama açısından anlamlı bir fark olmadığı tespit edildiği için
# average bidding ve maximum bidding teklif yöntemlerinin her ikisini de kullanabilir
